# CSUSB Travel Abroad Chatbot - Documentation

## **1. Introduction**
The CSUSB Travel Abroad Chatbot is a **Streamlit-based chatbot** that provides information related to travel abroad opportunities at **California State University, San Bernardino (CSUSB)**.

### Features:
- A **cooldown system** to limit excessive queries and prevent overloading the server.
- **Message persistence** using `st.session_state` to retain chat history.
- A **basic interactive UI** that allows users to input questions and receive responses.

## **2. Installation Requirements**
To run this chatbot, install Streamlit:
```bash
pip install streamlit
```
Other built-in Python modules used:
- `time`: Used for handling **cooldown timers** and response tracking.

## **3. Code Explanation**

### **3.1 Importing Required Libraries**
```python
import streamlit as st  # Streamlit for web-based chatbot UI
import time  # Time module for cooldown system
```
- `streamlit` is used to create the **interactive chatbot UI**.
- `time` is used to **track message cooldown periods** and measure response times.

### **3.2 Cooldown System Configuration**
```python
COOLDOWN_CHECK_PERIOD: float = 60.  # Time window (in seconds) for checking message frequency
MAX_MESSAGES_BEFORE_COOLDOWN: int = 10  # Maximum messages allowed before cooldown activates
COOLDOWN_DURATION: float = 180.  # Cooldown period in seconds (3 minutes)
```
- Users can send **10 messages per minute** before hitting a cooldown.
- Once the **limit is exceeded**, users must **wait 3 minutes** before sending more messages.

### **3.3 Cooldown Management Function**
```python
def canAnswer() -> bool:
    currentTimestamp = time.monotonic()  # Get the current timestamp
```
- `time.monotonic()` ensures that time **only moves forward**, preventing issues with time tracking.
- This function **determines if the chatbot can answer** based on the cooldown status.

```python
    if st.session_state["cooldownBeginTimestamp"] is not None:  # Check if cooldown is active
        if currentTimestamp - st.session_state["cooldownBeginTimestamp"] >= COOLDOWN_DURATION:
            st.session_state["cooldownBeginTimestamp"] = None  # Reset cooldown if time has passed
            return True  # Allow user to send a message
```
- If **cooldown is active**, it checks whether **enough time has passed**.
- If the cooldown period **has expired**, the chatbot resets the cooldown.

```python
    else:
        st.session_state["messageTimes"] = st.session_state["messageTimes"][-MAX_MESSAGES_BEFORE_COOLDOWN:]
        st.session_state["messageTimes"].append(currentTimestamp)  # Track message timestamps
```
- Stores only the last **10 message timestamps** (removes old ones to manage memory).
- Ensures that **only recent messages are considered** for cooldown checks.

```python
        if len(st.session_state["messageTimes"]) <= MAX_MESSAGES_BEFORE_COOLDOWN or            st.session_state["messageTimes"][-1] - st.session_state["messageTimes"][-MAX_MESSAGES_BEFORE_COOLDOWN - 1] >= COOLDOWN_CHECK_PERIOD:
            return True  # Allow message
```
- Checks if the user has sent fewer than **10 messages per minute**.
- If the limit is not reached, **they can continue chatting**.

```python
        else:
            st.session_state["cooldownBeginTimestamp"] = currentTimestamp  # Start cooldown
```
- **Activates cooldown mode** if the limit is reached.

```python
    remainingTime = COOLDOWN_DURATION + st.session_state["cooldownBeginTimestamp"] - currentTimestamp
    st.write(f"ERROR: You've reached the limit of {MAX_MESSAGES_BEFORE_COOLDOWN} messages. Please try again in {int(remainingTime//60)} minutes.")
    return False  # Prevent further messages
```
- **Calculates the remaining cooldown time** and **displays an error message** if the user must wait.

### **3.4 Setting Up Streamlit Chat UI**
```python
st.html("<h1 style='text-align:center; font-size:48px'>CSUSB Travel Abroad Chatbot</h1>")
```
- Displays the **chatbot title** in large, centered text for better UI.

### **3.5 Initializing Session Variables**
```python
if "messages" not in st.session_state or not isinstance(st.session_state["messages"], list):
    st.session_state["messages"] = []  # Store chat history
if "cooldownBeginTimestamp" not in st.session_state:
    st.session_state["cooldownBeginTimestamp"] = None  # Track cooldown start time
if "messageTimes" not in st.session_state:
    st.session_state["messageTimes"] = []  # Store message timestamps
```
- **Ensures session state variables are initialized**:
  - `messages`: Stores **previous chat messages**.
  - `cooldownBeginTimestamp`: Keeps **track of cooldown activation**.
  - `messageTimes`: Stores **timestamps of sent messages**.

### **3.6 Displaying Chat History**
```python
for message in st.session_state["messages"]:  # Loop through stored messages
    with st.chat_message(message["role"]):  # Display message
        st.markdown(message["content"])
```
- **Loops through stored messages** and displays them in the chat window.
- **Keeps past conversations visible**.

### **3.7 Handling User Input**
```python
prompt = st.chat_input("What is your question?")  # Capture user input
if prompt and canAnswer():  # Check cooldown before processing message
```
- `st.chat_input()` **creates an input box** for the user.
- Calls `canAnswer()` **to check if the user is allowed to send a message**.

```python
    st.chat_message("human").markdown(prompt)  # Display user input
    st.session_state["messages"].append({"role": "human", "content": prompt})  # Save message
```
- **Displays the user’s message** in the chat window.
- **Saves the message** to maintain chat history.

### **3.8 Processing AI Response**
```python
    responseStartTime = time.monotonic()  # Start response timer
    with st.chat_message("ai"):
        response = "[LLM response here]"  # Placeholder for AI-generated text
        responseEndTime = time.monotonic()  # End response timer
        st.markdown(response)  # Display AI response
        st.session_state["messages"].append({"role": "ai", "content": response})  # Save response
```
- **Tracks response time** for AI-generated messages.
- Uses a **placeholder AI response** (`"[LLM response here]"`) that can be replaced with an **LLM-generated response**.

```python
    if responseEndTime:  # Display response time for tracking
        st.write(f"*(Last response took {responseEndTime - responseStartTime:.4f} seconds)*")
```
- **Displays the response time** to track chatbot efficiency.

## **4. Summary**
✅ **Prevents message spam** with a cooldown system.  
✅ **Saves chat history** for a seamless conversation flow.  
✅ **Provides AI-generated responses** (can be expanded with OpenAI).  
✅ **Tracks response time** for performance analysis.  
